In [7]:
import struct
import os
# Open the file in binary mode

safetensor_file = "./net.safetensors"

def get_meta_size(file):
    with open(file, 'rb') as f:
        # Read the first 8 bytes
        bytes_data = f.read(8)
        # print them as uint8 integers, convert hex to decimal
        print(" ".join([str(x) for x in bytes_data]))
        # Unpack the bytes to a 64-bit unsigned integer
        meta_size = struct.unpack('Q', bytes_data)[0]
        return meta_size

print(get_meta_size(safetensor_file))

240 16 3 0 0 0 0 0
200944
